In [ ]:
import numpy as np 
import pandas as pd 

import glob
import gc

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [ ]:
#size=300,200


def prepare(data):
    if data== 'train':
        frag = glob.glob('../input/cassava-leaf-disease-classification/train_images/*')
        train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
        indexs = []
        array=[]
        y_=[]
        y=[]

        #pbar = ProgressBar()    
        for i in range(0,len(frag)):
            indexs =  np.append(indexs,frag[i].split('train_images/')[1])
            y.append(train[train['image_id']==indexs[i]]['label'].values)
            y_.append(y[i][0])
            
            image = Image.open(frag[i])
            image = crop_center(image,200,200)
            array.append(np.asarray(image))
        
        array = np.asarray(array)
            
        y_=np.asarray(y_)  #risparmio memoria usando 'sparse_categorical_crossentropy'
       
        #y_ = np.matrix(pd.get_dummies(y_)) #per usare 'categorical_crossentropy'

        return(array,y_)
    
    if data=='test':
        image = Image.open('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg')
        image = crop_Center(image,200,200)
        image = np.array(image)
        image = np.float32(image)
        image = np.expand_dims(image,axis=0)
    return(image)
        

In [ ]:
%%time
x,y = prepare('train')

gc.collect()

In [ ]:
#preprocessing.scale(x)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x,y, random_state=1, test_size=0.2)

In [ ]:
num_features = 32
num_labels=5
batch_size = 64

model =  Sequential()

#model.add(BatchNormalization())     
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(200,200,3)))  #300,200
#model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())

#model.add(Dense(2*num_features, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_features, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
%%time 
history = model.fit(X_train,y_train,epochs=25,verbose=1, validation_data= (X_val,y_val),batch_size = batch_size)

In [ ]:
history_frame = pd.DataFrame(history.history)

In [ ]:
history_frame.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.lineplot(x = range(0,len(history_frame)) , y = history_frame['loss'],label = 'Training Loss')
sns.lineplot(x = range(0,len(history_frame))  , y = history_frame['val_loss'],label='Validation Loss')

In [ ]:
plt.figure(figsize=(8,6))
sns.lineplot(x = range(0,len(history_frame)) , y = history_frame['accuracy'],label = 'Training accuracy')
sns.lineplot(x = range(0,len(history_frame))  , y = history_frame['val_accuracy'],label='Validation accuracy')

In [ ]:
test = prepare('test')

In [ ]:
valore = model.predict(test)

In [ ]:
def sub(valore):
    original = pd.DataFrame(valore).idxmax(axis=1)
    value =  original[0]
    submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
    submission['label'] = value 
    submission.to_csv('submission.csv', index=False)
    print(submission)
    print('  Finished!  ')

In [ ]:
sub(valore)

In [ ]:
frag = glob.glob('../input/cassava-leaf-disease-classification/train_images/*')